<a href="https://colab.research.google.com/github/elizabethavargas/Investing/blob/main/BestDayoftheWeek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Best Day of the Week to Invest
Is there a best day of the week to invest in the S&P 500?

In [ ]:
# imports
import pandas as pd
import yfinance as yf

In [ ]:
def get_stock_data(ticker, start, end):
    # Fetch historical data using yfinance
    sp500_data = yf.download(ticker, start=start, end=end)
    return sp500_data

In [ ]:
# get data
sp500_data = get_stock_data('^GSPC', '2018-07-31', '2023-07-31') # S&P 500 index
sp500_data = pd.DataFrame(sp500_data)
sp500_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-07-31,2809.729980,2824.459961,2808.060059,2816.290039,2816.290039,3906890000
2018-08-01,2821.169922,2825.830078,2805.850098,2813.360107,2813.360107,3501350000
2018-08-02,2800.479980,2829.909912,2796.340088,2827.219971,2827.219971,3501380000
2018-08-03,2829.620117,2840.379883,2827.370117,2840.350098,2840.350098,3043690000
2018-08-06,2840.290039,2853.290039,2835.979980,2850.399902,2850.399902,2879020000
...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000


In [ ]:
# split the data to allow for prediction
sp500_train = sp500_data[sp500_data.index < '2023-01-01'].copy()
sp500_test = sp500_data[sp500_data.index >= '2023-01-01'].copy()

## Observing Means
Monday has the lowested opening, high, low, close, and adj close price. Tuesday has the highest (except for high).

In [ ]:
# add day of week column
sp500_train['Day of Week'] = sp500_train.index.day_name()

# means for each day
sp500_train.groupby('Day of Week').mean()

,Open,High,Low,Close,Adj Close,Volume
Day of Week,,,,,,
Friday,3539.265876,3560.259414,3513.218261,3539.251390,3539.251390,4.436451e+09
Monday,3531.872102,3552.369760,3508.195161,3532.351865,3532.351865,4.205004e+09
Thursday,3542.108932,3566.302256,3516.980442,3543.990401,3543.990401,4.321246e+09
Tuesday,3552.953839,3573.684889,3528.854018,3551.100520,3551.100520,4.299760e+09
Wednesday,3550.195744,3574.163338,3527.365045,3550.422455,3550.422455,4.308669e+09


### Buy on Wednesday
I currently invest every Wednesday, but is that ideal?

In [ ]:
day_invest = 100

# initalize invested and total columns
sp500_data_wed = sp500_test.copy()
sp500_data['Invested'] = 0
sp500_data['Total'] = 0

invested = 0
num_shares = 0
for index, row in sp500_data_wed.iterrows():
  # invest
  if index.weekday() == 2 and index > pd.to_datetime('2023-01-06') and index <= pd.to_datetime('2023-01-12'):
    num_shares += (day_invest / row['Open'])
    invested += day_invest
    day_invest = 100
  else:
    day_invest += 100

  sp500_data_wed.at[index, 'Invested'] = invested
  sp500_data_wed.at[index, 'Total'] = num_shares * row['Open']

# calculate return
sp500_data_wed['Return'] = round((sp500_data_wed['Total'] - sp500_data_wed['Invested']) / sp500_data_wed['Invested'], 3)
sp500_data_wed

,Open,High,Low,Close,Adj Close,Volume,Day of Week,Invested,Total,Return
Date,,,,,,,,,,
2023-01-03,3853.290039,3878.459961,3794.330078,3824.139893,3824.139893,3959140000,Tuesday,0.0,0.000000,NaN
2023-01-04,3840.360107,3873.159912,3815.770020,3852.969971,3852.969971,4414080000,Wednesday,0.0,0.000000,NaN
2023-01-05,3839.739990,3839.739990,3802.419922,3808.100098,3808.100098,3893450000,Thursday,0.0,0.000000,NaN
2023-01-06,3823.370117,3906.189941,3809.560059,3895.080078,3895.080078,3923560000,Friday,0.0,0.000000,NaN
2023-01-09,3910.820068,3950.570068,3890.419922,3892.090088,3892.090088,4311770000,Monday,0.0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000,Tuesday,700.0,804.924732,0.150
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000,Wednesday,700.0,812.417258,0.161
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000,Thursday,700.0,810.727895,0.158


### Buy on Monday
If you constantly invested every Monday vs Wednesday, what will your return be?

In [ ]:
day_invest = 100

# initalize invested and total columns
sp500_data_mon = sp500_test.copy()
sp500_data['Invested'] = 0
sp500_data['Total'] = 0

invested = 0
num_shares = 0
for index, row in sp500_data_mon.iterrows():
  # invest
  if index.weekday() == 0 and index <= pd.to_datetime('2023-01-09'):
    num_shares += (day_invest / row['Open'])
    invested += day_invest
    day_invest = 100
  else:
    day_invest += 100

  sp500_data_mon.at[index, 'Invested'] = invested
  sp500_data_mon.at[index, 'Total'] = num_shares * row['Open']

# calculate return
sp500_data_mon['Return'] = round((sp500_data_mon['Total'] - sp500_data_mon['Invested']) / sp500_data_mon['Invested'], 3)
sp500_data_mon

,Open,High,Low,Close,Adj Close,Volume,Day of Week,Invested,Total,Return
Date,,,,,,,,,,
2023-01-03,3853.290039,3878.459961,3794.330078,3824.139893,3824.139893,3959140000,Tuesday,0.0,0.000000,NaN
2023-01-04,3840.360107,3873.159912,3815.770020,3852.969971,3852.969971,4414080000,Wednesday,0.0,0.000000,NaN
2023-01-05,3839.739990,3839.739990,3802.419922,3808.100098,3808.100098,3893450000,Thursday,0.0,0.000000,NaN
2023-01-06,3823.370117,3906.189941,3809.560059,3895.080078,3895.080078,3923560000,Friday,0.0,0.000000,NaN
2023-01-09,3910.820068,3950.570068,3890.419922,3892.090088,3892.090088,4311770000,Monday,500.0,500.000000,0.000
...,...,...,...,...,...,...,...,...,...,...
2023-07-18,4521.779785,4562.299805,4514.589844,4554.979980,4554.979980,4090010000,Tuesday,500.0,578.111458,0.156
2023-07-19,4563.870117,4578.430176,4557.479980,4565.720215,4565.720215,4115670000,Wednesday,500.0,583.492725,0.167
2023-07-20,4554.379883,4564.740234,4527.560059,4534.870117,4534.870117,3761770000,Thursday,500.0,582.279395,0.165
